In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Define the parameters of the bivariate distribution
mu1 = torch.tensor([0, 0], dtype=torch.float32)
mu2 = torch.tensor([2, 2], dtype=torch.float32)
mu3 = torch.tensor([-2, 2], dtype=torch.float32)
sigma = 2

# Generate 10,000 samples from the bivariate distribution
samples = np.concatenate([
    np.random.multivariate_normal(mu1, sigma * np.eye(2), 3333),
    np.random.multivariate_normal(mu2, sigma * np.eye(2), 3333),
    np.random.multivariate_normal(mu3, sigma * np.eye(2), 3334)
])


In [ ]:
# Define the generator and discriminator networks
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 128),
            nn.ReLU(),
            nn.Linear(128, 2)
        )

    def forward(self, z):
        return self.model(z)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
# Initialize the generator and discriminator
generator = Generator()
discriminator = Discriminator()

# Define the loss function and optimizers
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=0.001)
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.001)


# Question 2:

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision.utils as vutils
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
from inception_score import inception_score # from folder

In [3]:
torch.manual_seed(42)
if torch.backends.mps.is_available():
    device = torch.device("mps")
print("Device:", device)

Device: mps


In [5]:
# Define the generator and discriminator architectures
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )
        print("Ended Generator Code")

    def forward(self, input):
        return self.model(input)

In [6]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 128, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
        print("Ended Discriminator Code")

    def forward(self, input):
        return self.model(input)

In [7]:
# Initialize the generator and discriminator
generator = Generator()
discriminator = Discriminator()

# Define the loss function and optimizers
criterion = nn.BCELoss().to(device)
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Set up DataLoader for CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

batch_size = 64
dataset = datasets.CIFAR10(root='./data', download=True, train=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

Ended Generator Code
Ended Discriminator Code
Files already downloaded and verified


In [8]:
# Training the DCGAN
num_epochs = 5
generator.to(device)
discriminator.to(device)

for epoch in tqdm(range(num_epochs)):
    for i, data in enumerate(dataloader, 0):
        # Update discriminator with real data
        discriminator.zero_grad()
        real_images, _ = data
        real_images = real_images.to(device)
        batch_size_curr = real_images.size(0)  # Get the current batch size
        label = torch.full((batch_size_curr, 1, 1, 1), 1.0, device=device)
        output = discriminator(real_images)
        # errD_real = criterion(output, label)
        errD_real = criterion(output, label.expand_as(output))
        errD_real.backward()
        D_x = output.mean().item()
        print("Worked real output", i , "Times")
        
        # Update discriminator with fake data
        noise = torch.randn(batch_size_curr, 100, 1, 1, device=device)  # Using batch_size_curr instead of batch_size
        fake_images = generator(noise)
        label.fill_(0.0)
        output = discriminator(fake_images.detach())
        errD_fake = criterion(output, label.expand_as(output))
        print("errD_fake Criterion worked ", i , "times")
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizer_D.step()

        # Update generator
        generator.zero_grad()
        label.fill_(1.0)
        output = discriminator(fake_images)
        # print("Discriminator: ")
        errG = criterion(output, label.expand_as(output))
        # print("errG:")
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizer_G.step()
print("Train loop ended")     
  
# Save the generator model
torch.save(generator.state_dict(), 'generatorr.pth')

  0%|          | 0/5 [00:00<?, ?it/s]

Worked real output 0 Times
errD_fake Criterion worked  0 times
Worked real output 1 Times
errD_fake Criterion worked  1 times
Worked real output 2 Times
errD_fake Criterion worked  2 times
Worked real output 3 Times
errD_fake Criterion worked  3 times
Worked real output 4 Times
errD_fake Criterion worked  4 times
Worked real output 5 Times
errD_fake Criterion worked  5 times
Worked real output 6 Times
errD_fake Criterion worked  6 times
Worked real output 7 Times
errD_fake Criterion worked  7 times
Worked real output 8 Times
errD_fake Criterion worked  8 times
Worked real output 9 Times
errD_fake Criterion worked  9 times
Worked real output 10 Times
errD_fake Criterion worked  10 times
Worked real output 11 Times
errD_fake Criterion worked  11 times
Worked real output 12 Times
errD_fake Criterion worked  12 times
Worked real output 13 Times
errD_fake Criterion worked  13 times
Worked real output 14 Times
errD_fake Criterion worked  14 times
Worked real output 15 Times
errD_fake Criteri

 20%|██        | 1/5 [06:15<25:03, 375.91s/it]

In [ ]:
# Generate 10 images from the learned distribution and save them
generator.eval()
for i in range(10):
    noise = torch.randn(1, 100, 1, 1, device=device)
    fake_image = generator(noise).detach().cpu().squeeze().numpy()
    fake_image = (fake_image + 1) / 2.0  # Rescale values to [0, 1]
    plt.imsave(f'generated_image_{i+1}.png', np.transpose(fake_image, (1, 2, 0)))

In [ ]:
# Plot the Loss vs Epoch curve
plt.plot(range(num_epochs), errD.item(), label="Discriminator Loss")
plt.plot(range(num_epochs), errG.item(), label="Generator Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
# Calculate Inception Score on the generated images
fake_images_list = []
for i in range(500):  # Generate 500 images for a more accurate Inception Score
    noise = torch.randn(1, 100, 1, 1, device=device)
    fake_images_list.append(generator(noise).detach().cpu())
fake_images_tensor = torch.cat(fake_images_list, dim=0)
inception_mean, inception_std = inception_score(fake_images_tensor, 
                                                cuda=False, #cuda=True, 
                                                batch_size=32, resize=True, splits=1)

print(f"Inception Score: {inception_mean} ± {inception_std}")